In [31]:
import sys
sys.path.insert(0, '/homes/kazi/iondel')

from itertools import groupby
from collections import defaultdict
import pickle

import numpy as np
import beer


from bokeh.plotting import figure, show, output_notebook, gridplot
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label


output_notebook()

Loading BokehJS ...

In [2]:
with open('exp_dp/timit/aud_bigram_mbn_4g_hierarchical_dirichlet_process/final.mdl', 'rb') as f:
    model = pickle.load(f)
model

BigramPhoneLoop(
  (modelset): DynamicallyOrderedModelSet(
    (original_modelset): DynamicallyOrderedModelSet(
      (original_modelset): JointModelSet(
        (modelsets): ModuleList(
          (0): MixtureSet(
            (categoricalset): CategoricalSet(
              (weights): ConjugateBayesianParameter(prior=Dirichlet, posterior=Dirichlet)
            )
            (modelset): NormalSet(
              (means_precisions): ConjugateBayesianParameter(prior=NormalGamma, posterior=NormalGamma)
            )
          )
          (1): MixtureSet(
            (categoricalset): CategoricalSet(
              (weights): ConjugateBayesianParameter(prior=Dirichlet, posterior=Dirichlet)
            )
            (modelset): NormalSet(
              (means_precisions): ConjugateBayesianParameter(prior=NormalGamma, posterior=NormalGamma)
            )
          )
        )
      )
    )
  )
  (graph): <CompiledGraph>
  (categoricalset): SBCategoricalSet(
    (stickbreaking): ConjugateBayesian

In [29]:
mean = model.categoricalset.mean.detach().numpy()

fig = figure(x_range=(0, len(mean)), y_range=(0, len(mean)))
dh, dw = mean.shape
fig.image(image=[mean], x=0, y=0, dh=dh, dw=dw)
show(fig)

In [12]:
hyp = {}
with open('exp_dp/timit/aud_bigram_mbn_4g_hierarchical_dirichlet_process/decode_perframe/full/trans', 'r') as f:
    for line in f:
        tokens = line.strip().split()
        uttid, trans = tokens[0], tokens[1:]
        hyp[uttid] = [x[0] for x in groupby(trans)]

In [35]:
counts = defaultdict(lambda: defaultdict(int))
for utt, trans in hyp.items():
    for i, val in enumerate(trans[1:], start=1):
        counts[trans[i - 1]][val] += 1

matrix = np.zeros((101, 101))
for context in counts:
    for unit in counts[context]:
        matrix[au.index(context), au.index(unit)] = counts[context][unit]
matrix /= matrix.sum(axis=1, keepdims=True)

fig1 = figure(x_range=(0, len(mean)), y_range=(0, len(mean)))
dh, dw = matrix.shape
fig1.image(image=[matrix], x=0, y=0, dh=dh, dw=dw)

fig2 = figure(x_range=(0, len(mean)), y_range=(0, len(mean)))
dh, dw = mean.shape
fig2.image(image=[mean], x=0, y=0, dh=dh, dw=dw)
show(gridplot([[fig1, fig2]]))

/mnt/matylda5/iondel/anaconda3/envs/beer/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/mnt/matylda5/iondel/anaconda3/envs/beer/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [55]:
#with open('exp_dp/timit/aud_bigram_mbn_4g_hierarchical_dirichlet_process/final.mdl', 'rb') as f:
with open('exp_dp/timit/aud_mbn_4g_dirichlet_process/final.mdl', 'rb') as f:
    model = pickle.load(f)
    
starts, ends = model.start_pdf, model.end_pdf
starts_idxs = [val for val in starts.values()]
end_idxs = [val for val in ends.values()]
A = model.graph.trans_log_probs
A[:, starts_idxs][end_idxs, :]

tensor([[ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281],
        [ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281],
        [ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281],
        ...,
        [ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281],
        [ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281],
        [ -4.1607,  -6.2914,  -6.0375,  ...,  -6.3630, -14.1445,  -5.9281]])

In [65]:
with open('exp_dp/timit/aud_bigram_mfcc_4g_hierarchical_dirichlet_process/final.mdl', 'rb') as f:
    model = pickle.load(f)
model._on_weights_update()

with open('exp_dp/timit/aud_bigram_mfcc_4g_hierarchical_dirichlet_process/final.mdl', 'wb') as f:
    pickle.dump(model, f)

In [64]:
pickle.dump?


Signature: pickle.dump(obj, file, protocol=None, *, fix_imports=True)
Docstring:
Write a pickled representation of obj to the open file object file.

This is equivalent to ``Pickler(file, protocol).dump(obj)``, but may
be more efficient.

The optional *protocol* argument tells the pickler to use the given
protocol supported protocols are 0, 1, 2, 3 and 4.  The default
protocol is 3; a backward-incompatible protocol designed for Python 3.

Specifying a negative protocol version selects the highest protocol
version supported.  The higher the protocol used, the more recent the
version of Python needed to read the pickle produced.

The *file* argument must have a write() method that accepts a single
bytes argument.  It can thus be a file object opened for binary
writing, an io.BytesIO instance, or any other custom object that meets
this interface.

If *fix_imports* is True and protocol is less than 3, pickle will try
to map the new Python 3 names to the old module names used in Python
2, s